# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_for_csv = "../WeatherPy/output_data/city_weather.csv"
weather_df_uncleaned = pd.read_csv(weather_for_csv)

# weather_df_uncleaned.dtypes (change humidity to float)
weather_df_uncleaned

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Hithadhoo,-0.6000,73.0833,82.94,75,61,13.18,MV,1609912661
1,Tiksi,71.6872,128.8694,-22.56,85,84,21.23,RU,1609912662
2,Viedma,-40.8135,-62.9967,58.89,83,78,6.38,AR,1609912662
3,Jamestown,42.0970,-79.2353,32.00,92,90,5.82,US,1609912662
4,Corat,40.5725,49.7064,42.80,81,40,9.17,AZ,1609912663
...,...,...,...,...,...,...,...,...,...
635,Sistranda,63.7256,8.8340,28.40,86,0,17.22,NO,1609912904
636,Constitución,-35.3333,-72.4167,54.88,84,0,1.90,CL,1609912904
637,Roald,62.5833,6.1333,33.80,64,75,8.05,NO,1609912904
638,Makokou,0.5738,12.8642,71.10,94,99,1.34,GA,1609912905


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

# set up lat/lng as locs and hum as weight
locations = weather_df_uncleaned[["Lat", "Lng"]]
humidity = weather_df_uncleaned["Humidity"]

In [4]:
# set up heatmap then heatlayer to add over map
heatmap = gmaps.figure(center=(20, 10), zoom_level=2)

heatlayer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius=2.5)

heatmap.add_layer(heatlayer)
heatmap

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
best_weather_df = weather_df_uncleaned

best_weather_df = best_weather_df.loc[(best_weather_df["Max Temp"] <= 85) & (best_weather_df["Max Temp"] > 72)]
best_weather_df = best_weather_df.loc[best_weather_df["Wind Speed"] <= 5]
best_weather_df = best_weather_df.loc[best_weather_df["Cloudiness"] <= 30]

best_weather_df = best_weather_df.dropna()
best_weather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
45,Caravelas,-17.7125,-39.2481,72.99,90,5,4.45,BR,1609912679
154,Cockburn Town,21.4612,-71.1419,76.39,55,0,2.82,TC,1609912720
179,Itarema,-2.9248,-39.9167,80.60,74,0,2.24,BR,1609912727
187,Tautira,-17.7333,-149.1500,82.40,69,20,4.70,PF,1609912728
209,Rājāpur,25.3833,81.1500,81.57,34,5,4.25,IN,1609912735
241,Lambaré,-25.3468,-57.6065,77.00,88,0,2.24,PY,1609912744
251,George Town,5.4112,100.3354,82.40,74,20,3.36,MY,1609912687
280,Ashta,23.0167,76.7167,79.34,35,11,2.28,IN,1609912754
357,Keti Bandar,24.1447,67.4497,72.57,25,0,3.27,PK,1609912779
403,The Valley,18.2170,-63.0578,75.99,83,20,4.70,AI,1609912793


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = best_weather_df

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# set params before for loop
params = {"radius": 5000, 
          "type": "lodging", 
          "key": g_key,}

# loop thru last df, get lat and lng, create json response, 
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    
    # params['location'] will allow loop to find specific lats/lngs for each loop iteration (nearest restr. activity)
    params['location'] = f"{lat}, {lng}"
    
    # set json resp.
    response = requests.get(base_url, params=params).json()
    results = response['results']
    
    # now place results into a new column "Hotel Name"
    
    try:
        print(f"Closest hotel is {results[0]['name']} for City {index}: {row['City']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        print("----------")

    except:
        print("Sorry, result can not be found... Moving to next City.")

Closest hotel is Pousada dos Navegantes for City 45: Caravelas.
----------
Closest hotel is Osprey Beach Hotel for City 154: Cockburn Town.
----------
Closest hotel is Windvillage for City 179: Itarema.
----------
Closest hotel is Pension Oaoa for City 187: Tautira.
----------
Closest hotel is B.K. Computer & Janseva Kendra for City 209: Rājāpur.
----------
Closest hotel is Villa Floreal Hotel Boutique for City 241: Lambaré.
----------
Closest hotel is Cititel Penang for City 251: George Town.
----------
Closest hotel is Digambar Jain Dharamshala for City 280: Ashta.
----------
Closest hotel is Haji laloo memon house for City 357: Keti Bandar.
----------
Closest hotel is CeBlue Villas & Beach Resort for City 403: The Valley.
----------
Closest hotel is Pin Drop Hotel for City 442: Anloga.
----------
Closest hotel is HS Hotsson Smart Acapulco for City 482: Acapulco de Juárez.
----------
Closest hotel is Flat Hotel La Benediction for City 558: Basoko.
----------
Closest hotel is City Lin

In [7]:
hotel_df = hotel_df.dropna()
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
45,Caravelas,-17.7125,-39.2481,72.99,90,5,4.45,BR,1609912679,Pousada dos Navegantes
154,Cockburn Town,21.4612,-71.1419,76.39,55,0,2.82,TC,1609912720,Osprey Beach Hotel
179,Itarema,-2.9248,-39.9167,80.60,74,0,2.24,BR,1609912727,Windvillage
187,Tautira,-17.7333,-149.1500,82.40,69,20,4.70,PF,1609912728,Pension Oaoa
209,Rājāpur,25.3833,81.1500,81.57,34,5,4.25,IN,1609912735,B.K. Computer & Janseva Kendra
241,Lambaré,-25.3468,-57.6065,77.00,88,0,2.24,PY,1609912744,Villa Floreal Hotel Boutique
251,George Town,5.4112,100.3354,82.40,74,20,3.36,MY,1609912687,Cititel Penang
280,Ashta,23.0167,76.7167,79.34,35,11,2.28,IN,1609912754,Digambar Jain Dharamshala
357,Keti Bandar,24.1447,67.4497,72.57,25,0,3.27,PK,1609912779,Haji laloo memon house
403,The Valley,18.2170,-63.0578,75.99,83,20,4.70,AI,1609912793,CeBlue Villas & Beach Resort


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:
# Add marker layer ontop of heat map
# markerlayer = gmaps.marker_layer(locations)
marker = gmaps.marker_layer(locations, info_box_content=hotel_info,)
heatmap.add_layer(marker)

# Display figure
heatmap

Figure(layout=FigureLayout(height='420px'))